# Imports

In [ ]:
import dft_toolbox as dft
import numpy as np
import scipy.stats as st
import glob
import matplotlib.pyplot as plt

# Create G16 Input Files

Variables

In [ ]:
nodes = 5
mem = 91
partition = "mpiqdr"

GasRouteSection = "rb3lyp/aug-cc-pvdz empiricaldispersion=gd3 int(grid=ultrafine, acc2e=11) scf=(tight,maxcycles=500) iop(1/152=1000) opt=(maxcycles=1000,cartesian) freq=noraman nosymm"
# opt=(maxstep=1) iop(1/8=1)
PCMRouteSection = "geom=check rb3lyp-aug-cc-pvdz empiricaldispersion=gd3 int(grid=ultrafine, acc2e=11) scf=(tight,maxcycles=500) scrf=(iefpcm,solvent=water,externaliteration,1stvac,read) nosymm"

Loop through files

In [ ]:
coordFiles = glob.glob("*.xyz")
for num, coordFile in enumerate(coordFiles):
  dft.create_g16_input(
    fname=f"sim{num:03}",
    GasRouteSection=GasRouteSection,
    PCMRouteSection=PCMRouteSection,
    coordFile=coordFile
  )
  dft.create_slurm_script(
    fname=f"sim{num:03}",
    nodes=nodes,
    mem=mem,
    partition=partition
  )

# Create Arkane Input Files

Loop through files

In [ ]:
freqOutput = glob.glob("*_gas.log")
PCMOutput = glob.glob("*_PCM.log")

for i in range(len(freqOutput)):
    dft.create_arkane_input(
      f"sim{i:03}",
      freq_log=freqOutput[i],
      #pcm_log=PCMOutput[i],
      linear=False,
      spinMultiplicity=1,
      opticalIsomers=1
    )

# pQCT Example

Preliminary variables

In [ ]:
n_water = [] # shape (N,)
temperatures = np.linspace(10, 50, num=20) + 273.15 # shape (20,)

Loop through temps and do gas-phase thermo calc

In [ ]:
# shape (20,)
G_water = []
H_water = []
S_water = []

# shape (20,)
G_Na = []
H_Na = []
S_Na = []

# shape (20,N)
G_clusters = []
H_clusters = []
S_clusters = []

In [ ]:
for temp in temperatures:
  # water
  values = dft.calc_thermo_Arkane("templates/refChem.inp", temp=temp)[1] # H2O
  G_water.append(values[3])
  H_water.append(values[1])
  S_water.append(values[2])
  
  # ion
  values = dft.calc_thermo_Arkane("templates/refChem.inp", temp=temp)[2] # Na+
  G_water.append(values[3])
  H_water.append(values[1])
  S_water.append(values[2])
  
  # cluster
  values = dft.calc_thermo_Arkane("templates/clustersChem.inp", temp=temp) # shape (N,4)
  values = np.transpose(values) # shape (4,N)
  G_clusters.append(values[3])
  H_clusters.append(values[1])
  S_clusters.append(values[2])

In [ ]:
G_water = np.array(G_water)
H_water = np.array(H_water)
S_water = np.array(S_water)    

G_Na = np.array(G_Na)
H_Na = np.array(H_Na)
S_Na = np.array(S_Na)

Extract PCM solvation energies

In [ ]:
cluster_PCM_dG = [dGSolvPCM(fname) for fname in PCMOutput]
H2O_PCM_dG = dGSolvPCM("H2O_PCM.log")

Free energy **in solution**

In [ ]:
G_aq_283K = [calc_pQCT(G_cluster[0][i], G_water[0], cluster_PCM_dG[0], n_water, dG_solv_H2O=H2O_PCM_dG, temp=temperatures[0])]

Free energy **of solvation**

In [ ]:
dG_solv_283K = G_aq_283K - G_Na[0] - (R * temperatures[0] * np.log(24.46)) # shape (N,)

Boltzmann Averaging and Bootstrapping

In [ ]:
R = 0.0019872042586408316  # kcal/(mol*K)
beta = [1/(R*temperatures[0]) for i in range(len(dG_solv_283K))]

In [ ]:
mid = boltzmannG(dG_solv_283K,beta)
bootstrappedCI = st.bootstrap((dG_solv_283K,beta),boltzmannG,paired=True,vectorized=False,n_resamples=10000)
lowerBar = bootstrappedCI.confidence_interval[1]-mid
upperBar = mid-bootstrappedCI.confidence_interval[0]